In [6]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os

credits = pd.read_csv("data/tmdb_5000_credits.csv")
movies = pd.read_csv("data/tmdb_5000_movies.csv")
movies = movies.merge(credits,on = 'title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.dropna(inplace=True)
movies.isnull().sum()
movies.duplicated().sum()
movies.iloc[0].genres

import ast

def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(2)
movies.head(2)
movies['cast'][0]

def convert_cast(text):
    L=[]
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
movies['cast']=movies['cast'].apply(convert_cast)
movies.head(2)

def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director' :
            L.append(i['name'])
            break
    return L
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])
#movies.head()
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
movies.head()
new_dataframe=movies[['movie_id','title','tags']]
new_dataframe['tags']=new_dataframe['tags'].apply(lambda x:" ".join(x))
new_dataframe['tags'][0]
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:x.lower())
#new_dataframe.head()

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)


new_dataframe['tags'].apply(stem)

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(max_features=5000,stop_words='english')
vectors = count.fit_transform(new_dataframe['tags']).toarray()
#count.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity[1]

def recommend_movie(movie):
    movie_index=new_dataframe[new_dataframe['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_dataframe.iloc[i[0]].title)
        
recommend_movie('Avatar')


C:\Users\91906\AppData\Local\Temp\ipykernel_7136\275703982.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataframe['tags']=new_dataframe['tags'].apply(lambda x:" ".join(x))
C:\Users\91906\AppData\Local\Temp\ipykernel_7136\275703982.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:x.lower())


Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [7]:
import pickle
pickle.dump(new_dataframe.to_dict(),open('movies_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))